In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.agents import initialize_agent
from langchain.tools import Tool, BaseTool
from pydantic.v1 import BaseModel, Field
# from langchain.agents import load_tools
from langchain.memory import ConversationBufferMemory

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [3]:
import math
from scipy.stats import norm

def rw_corp(Input_String: str) -> str:

    """ 
    calculates the risk weight from input parameters 
    PD, LGD, MATURITY, F_LARGE_FIN and SIZE which are fed into the formula in this particular order 
    """
    
    PD_s, LGD_s, MATURITY_s, SIZE_s, F_LARGE_FIN_s = Input_String.split(",")

    PD = float(eval(PD_s.split(":")[1].strip()))
    LGD = float(eval(LGD_s.split(":")[1].strip()))
    MATURITY = float(eval(MATURITY_s.split(":")[1].strip()))
    SIZE = float(eval(SIZE_s.split(":")[1].strip()))
    F_LARGE_FIN = (F_LARGE_FIN_s.split(":")[1].strip())
       
    pd_final = max(0.0003, PD)
    size_final = max(5, min(SIZE, 50))    
    r0 = (0.12 * (1.0 - math.exp(-50.0 * pd_final)) / (1.0 - math.exp(-50.0))) + \
        (0.24 * (1.0 - (1 - math.exp(-50.0 * pd_final)) / (1.0 - math.exp(-50.0)))) - \
        (0.04 * (1.0 - (size_final - 5.0) / 45.0))
    if F_LARGE_FIN == 'Y':
        r = r0 * 1.25
    else:
        r = r0
    b = (0.11852 - 0.05478 * math.log(pd_final)) ** 2
    ma = ((1 - 1.5 * b) ** -1) * (1 + (MATURITY - 2.5) * b)    
    rw = ((LGD * norm.cdf((1 - r) ** -0.5 * norm.ppf(pd_final) + (r / (1 - r)) ** 0.5 * norm.ppf(0.999)) - pd_final * LGD) * ma) * 12.5 * 1.06
    return rw  


class RWInput(BaseModel):
    Input_String: str = Field(description='This is a string that contains values for the input parameters \
                              PD, LGD, MATURITY, SIZE and F_LARGE_FIN which are fed into the formula in this particular order ')
    
RWTool = Tool.from_function(
    func=rw_corp,
    name="RWTool",
    description="""
    This is a custom tool that calculates the risk weight from a set of input parameters:
        PD - Probability of Default,
        LGD - Loss Given Default,
        MATURITY - Remaining maturity of the loan in years,
        SIZE - The size of the client in MEUR, usually this is the client's turnover 
        F_LARGE_FIN - If 'Y' the client is a Large Financial Institution,
        
        """,
    args_schema=RWInput
)

In [4]:
llm = ChatOpenAI(temperature=0, model="gpt-4")
# llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
db = SQLDatabase.from_uri("sqlite:///./portfolio_data.db",
                          include_tables=['corporate_portfolio']
)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)

SQLTool = Tool(
        name="db_query_tool",
        func=db_chain.run,
        description="useful for quering a database"
)

In [5]:
tools = [SQLTool, RWTool]

memory = ConversationBufferMemory(memory_key="chat_history")

agent = initialize_agent(
    agent='conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=3,
    memory=memory,
    handle_parsing_errors=True
)


# agent = initialize_agent(
#     tools, 
#     llm, 
#     agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
#     verbose=True, 
#     handle_parsing_errors=True
# )

In [27]:
# output = agent.run("What are total RWA and EAD in table corporate_portfolio?")
output = agent.run("What are total RWA and EAD for reporting date 31-12-2022?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: db_query_tool
Action Input: {"query": "SELECT SUM(RWA), SUM(EAD) FROM corporate_portfolio WHERE reporting_date = '31-12-2022'"}

> Entering new SQLDatabaseChain chain...
{"query": "SELECT SUM(RWA), SUM(EAD) FROM corporate_portfolio WHERE reporting_date = '31-12-2022'"}
SQLQuery:SELECT SUM("RWA"), SUM("EAD") FROM corporate_portfolio WHERE "REPORTING_DATE" = '2022-12-31'
SQLResult: [(85075812.12498187, 156657332.34921697)]
Answer:The total RWA and EAD on 31-12-2022 are 85075812.12 and 156657332.35 respectively.
> Finished chain.

Observation: The total RWA and EAD on 31-12-2022 are 85075812.12 and 156657332.35 respectively.
Thought:Do I need to use a tool? No
AI: The total Risk-Weighted Assets (RWA) for the reporting date 31-12-2022 is approximately 85,075,812.12 and the total Exposure At Default (EAD) is approximately 156,657,332.35.

> Finished chain.


In [28]:
print(output)

The total Risk-Weighted Assets (RWA) for the reporting date 31-12-2022 is approximately 85,075,812.12 and the total Exposure At Default (EAD) is approximately 156,657,332.35.


In [7]:
output = agent.run("Read the information in table corporate_portfolio for loan id 1439964799kgeq68 and for reporting date 31-12-2022")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: db_query_tool
Action Input: {"table": "corporate_portfolio", "query": {"loan_id": "1439964799kgeq68", "reporting_date": "31-12-2022"}}

> Entering new SQLDatabaseChain chain...
{"table": "corporate_portfolio", "query": {"loan_id": "1439964799kgeq68", "reporting_date": "31-12-2022"}}
SQLQuery:SELECT "LOAN_ID", "REPORTING_DATE", "PARTNER_ID", "PRODUCT", "STATUS", "EXPOSURE_DRAWN", "EXPOSURE_UNDRAWN", "EXPOSURE", "EAD", "EAD_COLL", "PD", "LGD", "SIZE", "MATURITY", "F_LARGE_FIN", "RW", "RWA" FROM corporate_portfolio WHERE "LOAN_ID" = '1439964799kgeq68' AND "REPORTING_DATE" = '2022-12-31' LIMIT 5
SQLResult: [('1439964799kgeq68', '2022-12-31', 'dninx621xv9731', 'CREDIT_CARD', 'Performing', 5141.569897, 4578.250143, 9719.82004, 7061.256068, 0.0, 0.0156611, 0.7026, 2.345659392, 1.41369863, 'N', 1.196209223, 8446.739634)]
Answer:The loan with ID '1439964799kgeq68' on the reporting date '31-12-2022' belongs to 

In [8]:
output = agent.run("What will be the RW if PD doubles")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: RWTool
Action Input: {"PD": 0.0313222, "LGD": 0.7026, "MATURITY": 1.41, "SIZE": 2.35, "F_LARGE_FIN": "N"}
Observation: 1.4629293690925356
Thought:Do I need to use a tool? No
AI: If the Probability of Default (PD) doubles, the new Risk Weight (RW) would be approximately 1.463.

> Finished chain.


In [20]:
print(output)

If the Probability of Default (PD) doubles, the new Risk Weight (RW) would be approximately 1.463.


In [21]:
output = agent.run("What about if LGD doubles")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: RWTool
Action Input: {"PD": 0.0156611, "LGD": 0.7026*2, "MATURITY": 1.41, "SIZE": 2.35, "F_LARGE_FIN": "N"}
Observation: 2.3911990032552444
Thought:Do I need to use a tool? No
AI: If the Loss Given Default (LGD) doubles, the new Risk Weight (RW) would be approximately 2.391.

> Finished chain.


In [22]:
print(output)

If the Loss Given Default (LGD) doubles, the new Risk Weight (RW) would be approximately 2.391.


In [23]:
output = agent.run("And if MATURITY becomes 6.23")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: RWTool
Action Input: {"PD": 0.0156611, "LGD": 0.7026, "MATURITY": 6.23, "SIZE": 2.35, "F_LARGE_FIN": "N"}
Observation: 1.9901793310985323
Thought:Do I need to use a tool? No
AI: If the maturity becomes 6.23, the new Risk Weight (RW) would be approximately 1.990.

> Finished chain.


In [24]:
print(output)

If the maturity becomes 6.23, the new Risk Weight (RW) would be approximately 1.990.


In [25]:
output = agent.run("What if all three parameters change like that?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: RWTool
Action Input: {"PD": 0.0313222, "LGD": 1.4052, "MATURITY": 6.23, "SIZE": 2.35, "F_LARGE_FIN": "N"}
Observation: 4.420645788063997
Thought:Do I need to use a tool? No
AI: If all three parameters change like that, the new Risk Weight (RW) would be approximately 4.421.

> Finished chain.


In [26]:
print(output)

If all three parameters change like that, the new Risk Weight (RW) would be approximately 4.421.


In [ ]:
output = agent.run("Start over again and read the all information in table corporate_portfolio for loan id 1439964799kgeq68 but for reporting date 2023-03-31")

In [ ]:
output = agent.run("What is the change in PD for loan id 1439964799kgeq68 between reporting date 2022-12-31 and 2023-03-31")

In [ ]:
output = agent.run("What is the other info for loan id 1439964799kgeq68 on reporting date 2023-03-31")

In [ ]:
output = agent.run("Read the information for ACCOUNT_NUMBER = '1851285872eypk20' in rwa_data table and then calculate the RW if LGD doubles and PD becomes 0.2")

In [ ]:
output = agent.run("Read the information for ACCOUNT_NUMBER = '1851285872eypk20' in rwa_data table")

In [ ]:
output = agent.run("What will be the RW if MATURITY is 5?")

In [ ]:
output = agent.run("What is the RW for LOAN_ID dninx621xv9731 for reporting date 31-12-2022?")

In [ ]:
output = agent.run("Read the info for LOAN_ID dninx621xv9731 for reporting date 31-12-2022")

In [ ]:
output = agent.run("What will be the RW if PD doubles?")

In [ ]:
output = agent.run("What is the difference between the sum of RWA for reporting date 2022-12-31 and the sum of RWA for reporting date 2023-03-31?")